<a href="https://colab.research.google.com/github/Akanksha131313/AI_Projects_Journey/blob/main/06_LSTM_Practical_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fake News Classifier Using Bidirectional LSTM -**

Dataset: https://www.kaggle.com/c/fake-news/data#

In [35]:
import pandas as pd

In [36]:
import numpy as np

In [37]:
df = pd.read_csv('/content/sample_data/Fake_news.csv', sep=',', on_bad_lines='skip', engine='python')

In [38]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [39]:
df.shape

(15966, 4)

In [40]:
#Finding NUll values
df.isnull().sum()

,0
Unnamed: 0,0
title,102
text,26
label,20


In [41]:
#Drop NaN Values :
df = df.dropna()

In [42]:
df.shape

(15843, 4)

In [43]:
df = df.reset_index(drop=True)

In [44]:
df

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
...,...,...,...,...
15838,15941,"Trump to offer views on law and order, judges ...",WASHINGTON (Reuters) - Presumptive Republican ...,0
15839,15942,"For Women in Advertising, It’s Still a ‘Mad Me...",As a bathroom break girl at the advertising ...,0
15840,15943,PRIORITIES: #BlackLivesMatter Terrorists Prote...,The December tally in Chicago (so far) is: Dec...,1
15841,15944,Comment on 15 Plants & Herbs That Can Boost Lu...,Share on Facebook Share on Twitter When it com...,1


In [45]:
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [46]:
# here Target ='Label'
# In label , there are mix values , check them first :

df['label'].apply(type).value_counts()

,count
label,
<class 'str'>,15843


In [47]:
import pandas as pd
import numpy as np

# Convert all label values to numeric — invalid ones become NaN
df['label'] = pd.to_numeric(df['label'], errors='coerce')

# Drop rows where label became NaN (non-numeric values)
df = df.dropna(subset=['label'])

# Convert final labels to int
df['label'] = df['label'].astype(int)

# Reset index
df = df.reset_index(drop=True)


/tmp/ipython-input-2626874810.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int)


### **Dependent & Independent Features -**

In [48]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [49]:
## Get the Dependent features
y=df['label']

In [50]:
X.shape

(15841, 3)

In [51]:
y.shape

(15841,)

In [52]:
import tensorflow as tf

In [53]:
tf.__version__

'2.19.0'

In [54]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [55]:
### Vocabulary size
voc_size = 500000  # ensure max index fits embedding


### For Onehot Representation , we are doing some basic steps before -

In [56]:
messages=X.copy()

In [57]:
messages=X.copy()

messages.reset_index(inplace=True)

In [58]:
messages['title'][1]

'UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO]'

In [59]:
messages

,index,Unnamed: 0,title,text
0,0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
1,1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
2,2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
3,3,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
4,4,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...,...,...
15836,15836,15941,"Trump to offer views on law and order, judges ...",WASHINGTON (Reuters) - Presumptive Republican ...
15837,15837,15942,"For Women in Advertising, It’s Still a ‘Mad Me...",As a bathroom break girl at the advertising ...
15838,15838,15943,PRIORITIES: #BlackLivesMatter Terrorists Prote...,The December tally in Chicago (so far) is: Dec...
15839,15839,15944,Comment on 15 Plants & Herbs That Can Boost Lu...,Share on Facebook Share on Twitter When it com...


### **Cleaning the Data -**

i) Stemming

ii) Stopwords

In [60]:
import nltk
import re
from nltk.corpus import stopwords

In [61]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [63]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [64]:
corpus[1]

'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video'

### **One-Hot Representation -**

In [65]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[141193,
  209905,
  80214,
  423388,
  486323,
  180265,
  380840,
  297026,
  226278,
  92863,
  30124,
  127771],
 [362885,
  136987,
  52539,
  172260,
  305805,
  70968,
  32444,
  45444,
  370227,
  435116,
  465198,
  413157,
  403049,
  127771],
 [200573,
  159610,
  125607,
  499744,
  46215,
  495446,
  216376,
  458113,
  146888,
  446691,
  315763,
  449483],
 [161154,
  477842,
  281597,
  413406,
  176157,
  300963,
  199135,
  239493,
  125775,
  224079,
  449299],
 [400344, 216376, 420830, 85705, 124155, 236480, 122463, 51091, 420830],
 [408545,
  388173,
  230228,
  154308,
  251819,
  134350,
  497126,
  158120,
  378662,
  252590,
  484118],
 [221522,
  408411,
  196644,
  194469,
  477842,
  312541,
  495446,
  324282,
  219184,
  127771],
 [30738,
  207136,
  445370,
  105176,
  354691,
  314121,
  39608,
  91056,
  314626,
  30738,
  70722,
  329104,
  232225,
  391376,
  127771],
 [466536, 435043, 359023, 221964, 444951, 309491, 130888, 435043],
 [444812, 144988

In [66]:
corpus[1]

'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video'

In [67]:
onehot_repr[1]

[362885,
 136987,
 52539,
 172260,
 305805,
 70968,
 32444,
 45444,
 370227,
 435116,
 465198,
 413157,
 403049,
 127771]

### **Embedding Representation -**

In [68]:
sent_length=100
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[141193 209905  80214 ...      0      0      0]
 [362885 136987  52539 ...      0      0      0]
 [200573 159610 125607 ...      0      0      0]
 ...
 [ 25245 226278  30124 ...      0      0      0]
 [489153 124826 201364 ...      0      0      0]
 [ 46403 376596 285983 ...      0      0      0]]


In [69]:
embedded_docs[1]

array([362885, 136987,  52539, 172260, 305805,  70968,  32444,  45444,
       370227, 435116, 465198, 413157, 403049, 127771,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0], dtype=int32)

In [70]:
embedded_docs[0]

array([141193, 209905,  80214, 423388, 486323, 180265, 380840, 297026,
       226278,  92863,  30124, 127771,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0], dtype=int32)

### **LSTM NN -**

In [73]:
## Creating model -> it's super important step
embedding_vector_features=80 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features ))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))# Add 128 layers f/d & b/d
model.add(Dropout(0.3)) # Drop-out layer
model.add(Dense(1,activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0) # set your own learning_rate

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


## build the model, provide a dummy shape (just for the summary)

model.build(input_shape=(None, sent_length))

print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 80)        │    40,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 80)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256)            │       214,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,214,273 (153.41 MB)

 Trainable params: 40,214,273 (153.41 MB)

 Non-trainable params: 0 (0.00 B)

None


In [74]:
len(embedded_docs),y.shape

(15841, (15841,))

In [75]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [76]:
X_final.shape,y_final.shape

((15841, 100), (15841,))

In [77]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [78]:
X_train = X_train.astype('float32')

### **Model Training -**

In [80]:
# Early Stoping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss',patience=20,verbose=1,mode='auto',baseline=None, restore_best_weights=False)

In [81]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=100 , callbacks=early_stopping)

Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.6523 - loss: 0.5811 - val_accuracy: 0.8514 - val_loss: 0.3174
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.8841 - loss: 0.2757 - val_accuracy: 0.8766 - val_loss: 0.2860
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9399 - loss: 0.1650 - val_accuracy: 0.8761 - val_loss: 0.3035
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.9665 - loss: 0.1015 - val_accuracy: 0.8695 - val_loss: 0.3523
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.9767 - loss: 0.0690 - val_accuracy: 0.8621 - val_loss: 0.4116
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.9856 - loss: 0.0434 - val_accuracy: 0.8684 - val_loss: 0.4451
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.9901 - loss: 0.0319 - val_accuracy: 0.8678 - val_loss: 0.5220
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.9890 - loss: 0.0324 - val_acc

### **Performance Metrics And Accuracy -**

In [82]:
y_pred=model.predict(X_test)

164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [83]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve #1 means it's fake & 0 means it's 'real'

In [84]:
from sklearn.metrics import confusion_matrix

In [85]:
confusion_matrix(y_test,y_pred)

array([[2072,  402],
       [ 303, 2451]])

In [86]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8651491966335119

In [87]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.84      0.85      2474
           1       0.86      0.89      0.87      2754

    accuracy                           0.87      5228
   macro avg       0.87      0.86      0.86      5228
weighted avg       0.87      0.87      0.86      5228

